<a href="https://colab.research.google.com/github/karim-mammadov/Computer-Vision-My-projects/blob/main/Flowers_Datasetsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download alsaniipe/flowers-dataset

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('//content/flowers-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
test_path="/content/test"
train_path='/content/train'
valid_path='/content/valid'

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
categories = os.listdir(train_path)

In [ ]:
for category in categories:
    image_path = os.path.join(train_path, category, os.listdir(os.path.join(train_path, category))[0])
    img = mpimg.imread(image_path)
    plt.figure(figsize=(3, 3))
    plt.imshow(img)
    plt.title(category)
    plt.axis('off')
    plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    seed=13,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical'
)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(
    valid_path,
    seed=13,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical'

)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    seed=13,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical'
    )

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
batch_size = 32
img_size = (224, 224)

datagen= ImageDataGenerator(rescale=1./255, validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

train_dir = '/content/train'
val_dir = '/content/test'

train_data = tf.keras.utils.image_dataset_from_directory(
    train_path,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode = 'binary')

valid_data = tf.keras.utils.image_dataset_from_directory(
    valid_path,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode = 'binary')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import regularizers

model = Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3), strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(64, (3,3), strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3,3), strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(256, (3,3), strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(512, (3,3), strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(1024, (1,1), strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')

])

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=valid_data
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [ ]:
model.fit(train_data, validation_data=valid_data, epochs=8, callbacks=[early_stop])